In [ ]:
pip install cohere flask pandas

In [ ]:
import cohere
import pandas as pd
from flask import Flask, request, jsonify

# Initialize Cohere client
cohere_client = cohere.Client("Your_ApiKey_Here")


In [ ]:
# Sample job postings
job_postings = [
    {"title": "Data Scientist", "description": "Analyze and interpret complex data to build AI models."},
    {"title": "AI Engineer", "description": "Develop AI systems and optimize pipelines."},
    {"title": "Backend Developer", "description": "Build and maintain server-side applications."},
    {"title": "DevOps Engineer", "description": "Ensure reliable deployment of applications and services."}
]

In [ ]:
# Function to analyze resumes
def analyze_resume(resume_text):
    # Get embeddings for the resume
    response = cohere_client.embed(texts=[resume_text])
    resume_embedding = response.embeddings[0]

    # Get embeddings for job postings
    job_descriptions = [job["description"] for job in job_postings]
    job_embeddings = cohere_client.embed(texts=job_descriptions).embeddings

    # Find the most relevant jobs using cosine similarity
    scores = [
        (job_postings[i]["title"], sum(a * b for a, b in zip(resume_embedding, job_embeddings[i])))
        for i in range(len(job_postings))
    ]
    scores.sort(key=lambda x: x[1], reverse=True)

    # Return the top 3 relevant jobs
    return [job[0] for job in scores[:3]]


In [ ]:
# Function to suggest skills
def suggest_skills(resume_text):
    prompt = f"The following resume belongs to a software engineer:\n\n{resume_text}\n\nSuggest five skills they should improve:"
    response = cohere_client.generate(
        model="command-xlarge-nightly",
        prompt=prompt,
        max_tokens=300,
        temperature=0.5,
    )
    return response.generations[0].text.strip().split("\n")

In [ ]:
# Interactive prompt for resume skills
def get_user_input():
    print("Enter your skill set and experience as a resume. For example:")
    print("""
    - Experienced in Python and Machine Learning.
    - Proficient in cloud platforms like AWS and Azure.
    - Developed scalable solutions for data analysis.
    - Collaborated with cross-functional teams.
    """)
    print("\nNow, input your own resume details:")
    resume_text = input("> ")
    return resume_text

# Main function
def main():
    resume_text = get_user_input()

    # Analyze the resume
    job_recommendations = analyze_resume(resume_text)
    skill_suggestions = suggest_skills(resume_text)

    # Display results
    print("\nRecommended Jobs:")
    for job in job_recommendations:
        print(f"- {job}")

    print("\nSkills to Learn:")
    for skill in skill_suggestions:
        print(f"- {skill}")

# Run the program
main()

Enter your skill set and experience as a resume. For example:

    - Experienced in Python and Machine Learning.
    - Proficient in cloud platforms like AWS and Azure.
    - Developed scalable solutions for data analysis.
    - Collaborated with cross-functional teams.
    

Now, input your own resume details:
> Experienced in Python and Machine Learnin

Recommended Jobs:
- AI Engineer
- Data Scientist
- Backend Developer

Skills to Learn:
- 1. Data Structures and Algorithms: A strong understanding of data structures such as arrays, linked lists, trees, graphs, and algorithms like sorting, searching, and dynamic programming is essential for software engineers. Improving these skills can enhance problem-solving abilities and code efficiency. 
- 
- 2. Database Management: Knowledge of relational databases (e.g., SQL), NoSQL databases (e.g., MongoDB), and database design principles is crucial. Software engineers should focus on improving their skills in querying, optimizing database perf